In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import bs4
import requests
from tqdm import tqdm
import re
import urllib

In [6]:
data = pd.read_csv("product_info.csv")
dupli = []
for i in data.columns:
    if i.endswith('_1'):
        dupli.append(i)

In [7]:
data2 = data.drop(dupli,axis=1)

In [12]:
data2 = data2.set_index(data2["Unnamed: 0"].values).drop("Unnamed: 0",axis=1)

In [132]:
links = data2.T["product_link"].values
names = data2.T["product_name"].values

In [16]:
len(links)

245

## Product Details

In [202]:
def get_product_details(links,names):
    
    product_details = {}
    for link , name in tqdm(zip(links,names)):
        
        req = urllib.request.Request(link)
        req.add_header('User-agent', "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")
        content = urllib.request.urlopen(req).read()
        soup  = BeautifulSoup(content , "html.parser")
        
        try:
            descp = ".".join([lst_tag.text.strip() for lst_tag in soup.find(id = 'feature-bullets').find_all(class_ = "a-list-item")])
        except:
            descp = None
        try:
            if soup.find(id="detailBulletsWrapper_feature_div") != None:
                for i in soup.find(id="detailBulletsWrapper_feature_div").find_all('li'):
                    try: 
                        if i.find(class_="a-text-bold").text.split("\n")[0].strip() == 'Manufacturer':
                            manuf = i.text.split("\n")[-1].strip()
                        elif i.find(class_="a-text-bold").text.split("\n")[0].strip() == 'ASIN':
                            asin = i.text.split("\n")[-1].strip()
                            break
                    except: 
                        continue
            
            elif soup.find(id = "productDetails_feature_div") != None:
                for i in soup.find(id = "productDetails_feature_div").find_all("tr"):
                    try:
                        if i.th.text.strip() == 'Manufacturer':
                            manuf = i.td.text.strip().encode("ASCII",'ignore')
                        elif i.th.text.strip() == 'ASIN':
                            asin = i.td.text.strip().encode("ASCII",'ignore')
                    except:
                        continue
                            
        except:
            manuf = "not_provided"
            asin = "not_provided"
                
        try:
            if soup.find(id = 'productDescription') != None:
                prod_descp = soup.find(id = 'productDescription').text.strip()
            elif soup.find(class_ = "aplus-v2 desktop celwidget") != None:
                prod_descp = "".join([txt.strip() for txt in soup.find(class_ = "aplus-v2 desktop celwidget").text.strip().replace('\n',"").split("  ")])
            else:
                prod_descp = "not_provided"
        except:
            prod_descp = "not_provided"
        
        info = {}
        info["product_name"] = name
        info['description'] = descp
        info['product_manufacturer'] = manuf
        info["ASIN"] = asin
        if len(prod_descp)>0:
            info['prod_description'] = prod_descp
        else:
            info['prod_description'] = "not_provided"
        
        product_details[name] = info
    return product_details

In [203]:
product_details = get_product_details(links , names)

245it [17:05,  4.19s/it]


In [206]:
prod_detail = pd.DataFrame(data = product_details).T

In [224]:
prod_detail2 = prod_detail.copy()

In [227]:
prod_detail2["ASIN"] = prod_detail2["ASIN"].apply(lambda x: x.decode() if type(x)==bytes else x)
prod_detail2["product_manufacturer"] = prod_detail2["product_manufacturer"].apply(lambda x: x.decode() if type(x)==bytes else x)

In [235]:
prod_detail2["url"] = links

In [236]:
prod_detail2.head()

,product_name,description,product_manufacturer,ASIN,prod_description,url
"MOKOBARA Backpack 15.5"" Inch Laptop Backpack For Women (16 litres ) Improved (2022 - Model)","MOKOBARA Backpack 15.5"" Inch Laptop Backpack F...",Care Instructions: Wipe with Damp Cloth.CRAFTE...,Designed By Mokobara Lifestyle Private Limited,B0BR7N3KF4,not_provided,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
MOKOBARA The Transit Backpack Laptop Bag for Men and Women,MOKOBARA The Transit Backpack Laptop Bag for M...,Care Instructions: Wipe with Damp Cloth.CRAFTE...,Designed By Mokobara Lifestyle Private Limited...,B09V7R98XC,Our Most Functional backpack EverThe Mokobara ...,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
Wesley Milestone 2.0 Casual Waterproof Laptop Backpack/Office Bag/School Bag/College Bag/Business Bag/Travel Backpack (Dimensions:13x18 inches) (Compatible with 39.62cm(15.6inch laptop) 30 L (Blue & black),Wesley Milestone 2.0 Casual Waterproof Laptop ...,30L Capacity: The Backpack has a padded laptop...,Wesley,B084JGJ8PF,"Ergonomic Contoured ShapeWith a slim, sleek an...",https://www.amazon.in/Wesley-Milestone-Waterpr...
American Tourister 32 Ltrs Black Casual Backpack (AMT FIZZ SCH BAG 02 - BLACK),American Tourister 32 Ltrs Black Casual Backpa...,"Laptop Compatibility: No, Strap Type: Adjustab...",Samsonite,B07CJCGM1M,not_provided,https://www.amazon.in/American-Tourister-AMT-S...
Skybags Brat Black 46 Cms Casual Backpack,Skybags Brat Black 46 Cms Casual Backpack,Combination of functional & safety features in...,VIP Industries Ltd,B08Z1HHHTD,Ideal for a college student who does not carry...,https://www.amazon.in/Skybags-Brat-Black-Casua...


In [237]:
prod_detail2.to_csv("product_details2.csv")

In [238]:
prod_detail.to_csv("product_details.csv")

In [239]:
prod_detail2.to_csv("product_details3.csv",index = False)